In [2]:
from datetime import datetime
from operator import itemgetter
from os import path

from astropy.table import Table
import numpy as np
from skynet.api.v20 import errors
from skynet.api.v20.methods import download, groups, obs, scopes

In [3]:
group_id = groups.get("Majewski Research Group")["id"]
scope_id = scopes.query(include="*", **{"name": "RRRT"})[0]["id"]
group_id, scope_id

(816, 27)

In [6]:
# make a list of all observations that match the observation_criteria defined below

observation_fields = {
    "id": "Observation ID",
    "state": "Observation state",
    "priority": "Priority",
    "name": "Target Name",
    "user.username": "Username",
    "group.name": "Group",
    "raHours": "RA",
    "decDegs": "Dec",
    "currentTelescope.name": "Telescope",
    "teleOwner.name": "Scope Owner",
    "timeIn": "Submitted",
}

start_time = "2024-08-15 00:00:00" # only get data from this time forward
observation_criteria = {"groupId": group_id}
# criteria = {"id": 3702107}
observation_results = obs.query(include=",".join(observation_fields.keys()), after=start_time, **observation_criteria)

observations = Table()
for field, col_name in observation_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 1:
            observations[col_name] = list(map(itemgetter(field), observation_results))
        case 2:
            field_dicts = list(map(itemgetter(field_parts[0]), observation_results))
            observations[col_name] = [field_dict[field_parts[1]] if field_dict is not None else "" for field_dict in field_dicts]
observations.write("skynet_observations.csv", overwrite=True)

observations

Observation ID,Observation state,Priority,Target Name,Username,Group,RA,Dec,Telescope,Scope Owner,Submitted
int32,str9,int32,str13,str7,str23,float64,float64,str4,str3,str19
11593072,completed,1,tic 344541836,gnh3pgz,Majewski Research Group,21.19002777777778,57.62038888888889,RRRT,UVA,2024-08-24T02:57:25
11593073,completed,1,TIC 278465736,gnh3pgz,Majewski Research Group,21.653363888888887,51.34663888888889,RRRT,UVA,2024-08-24T03:31:08
11593074,canceled,1,tic161043618,gnh3pgz,Majewski Research Group,14.895,52.715833333333336,RRRT,UVA,2024-08-24T03:45:17
11593075,completed,1,tic161043618,gnh3pgz,Majewski Research Group,14.895,52.715833333333336,RRRT,UVA,2024-08-24T03:48:15
11593076,completed,1,tic161043618,gnh3pgz,Majewski Research Group,14.895,52.715833333333336,RRRT,UVA,2024-08-24T03:51:26
11593786,completed,1,tic 344541836,gnh3pgz,Majewski Research Group,21.19002777777778,57.62038888888889,RRRT,UVA,2024-08-24T04:00:08
11594178,completed,1,tic 278465736,gnh3pgz,Majewski Research Group,21.65336111111111,51.34663888888889,RRRT,UVA,2024-08-24T04:09:57
11594179,completed,1,tic 278465736,gnh3pgz,Majewski Research Group,21.65336111111111,51.34663888888889,RRRT,UVA,2024-08-24T04:26:31
11594180,completed,1,tic367448265,gnh3pgz,Majewski Research Group,5.225472222222223,35.65302777777778,RRRT,UVA,2024-08-24T04:47:00


In [7]:
# make a list of all _exposures_ for the exposure_criteria defined below (using the start_time from above)

exposure_fields = {
    "exps.obsId": "Observation ID",
    "exps.expNum": "Exposure Num",
    "exps.imageId": "Image ID",
    "exps.obs.name": "Name",
    "exps.type": "Type",
    "exps.filterName": "Filter Name",
    "exps.timeTaken": "Time Taken",
    "exps.state": "Image State",
    "exps.expLengthUsed": "Exposure",
    "exps.binningUsed": "Binning",
    "exps.gain": "Gain",
    "exps.camera.name": "Camera",
}

exposure_criteria = {"groupId": group_id}
# exposure_criteria = {"id": 3702107}
# exposure_criteria = {"id": 10408766}

exposure_results = obs.query(include=",".join(exposure_fields.keys()), after=start_time, **exposure_criteria)

exp_structs = [image for result in exposure_results for image in result["exps"]]
images = Table()
for field, col_name in exposure_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 2:
            images[col_name] = list(map(itemgetter(field_parts[1]), exp_structs))
        case 3:
            field_dicts = list(map(itemgetter(field_parts[1]), exp_structs))
            images[col_name] = [field_dict[field_parts[2]] if field_dict is not None else "" for field_dict in field_dicts]
images.write("skynet_images.csv", overwrite=True)

images

Observation ID,Exposure Num,Image ID,Name,Type,Filter Name,Time Taken,Image State,Exposure,Binning,Gain,Camera
int32,int32,str9,str13,str5,str1,object,str8,object,int32,object,str40
11593072,0,r43471119,tic 344541836,light,V,2024-08-24T03:31:00,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593072,1,r43471120,tic 344541836,light,V,2024-08-24T03:31:25,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593072,2,r43471121,tic 344541836,light,V,2024-08-24T03:31:51,archived,30.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593072,3,r43471122,tic 344541836,light,V,2024-08-24T03:32:35,archived,30.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593072,4,r43471123,tic 344541836,light,V,2024-08-24T03:33:20,archived,60.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593072,5,r43471124,tic 344541836,light,V,2024-08-24T03:34:31,archived,60.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593073,0,r43471152,TIC 278465736,light,V,2024-08-24T03:35:50,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593073,1,r43471153,TIC 278465736,light,V,2024-08-24T03:36:10,archived,10.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel
11593073,2,r43471154,TIC 278465736,light,V,2024-08-24T03:36:34,archived,30.0,2,1.32256,UVA SBIG STX-16803 with FLI Filter Wheel


In [5]:
# can't remember what this part does

images_by_observation = images[images[exposure_fields["exps.state"]] == "archived"].group_by(exposure_fields["exps.obsId"])

# np.min(images_by_observation[exposure_fields["exps.timeTaken"]])

for observation_id, images in zip(images_by_observation.groups.keys, images_by_observation.groups):
    print(observation_id, len(images), np.min(images[exposure_fields["exps.timeTaken"]]), np.max(images[exposure_fields["exps.timeTaken"]]))

Observation ID
--------------
       3702107 12 2019-03-13T03:29:19 2019-03-13T07:52:25
Observation ID
--------------
       7484500 671 2022-03-15T00:24:19 2022-03-16T03:39:52
Observation ID
--------------
       7505658 106 2022-03-19T05:53:54 2022-03-19T09:36:20
Observation ID
--------------
       7514494 804 2022-03-21T03:50:40 2022-03-21T06:58:11
Observation ID
--------------
       7514505 334 2022-03-22T04:52:06 2022-03-22T10:12:52
Observation ID
--------------
       7514508 183 2022-03-25T04:50:09 2022-03-25T09:06:41
Observation ID
--------------
      10067782 268 2023-09-19T03:42:09 2023-09-19T09:59:05
Observation ID
--------------
      10067789 297 2023-09-20T00:15:17 2023-09-20T07:03:55
Observation ID
--------------
      10279223 2 2023-11-11T08:12:16 2023-11-11T08:12:41
Observation ID
--------------
      10286988 1 2023-11-12T02:38:30 2023-11-12T02:38:30
Observation ID
--------------
      10287062 2 2023-11-12T04:58:00 2023-11-12T04:58:49
Observation ID
-------------

In [6]:
# do the (sometimes slow) download of the identified images

for observation in observations:
    obs_id = observation[observation_fields["id"]]
    target = observation[observation_fields["name"]].strip().replace(" ", "_")
    filename = f"{obs_id}-{target}.zip"
    if path.isfile(filename):
        print(f"Observation {obs_id:10d} already downloaded")
        continue
    print(f"Downloading observation {obs_id:10d}...", end=" ")
    start = datetime.now()
    try:
        _, data = download.fits(**{"obs": obs_id})
    except errors.BadRequestError:
        print("skipped - got bad request")
        continue
    filesize = len(data)
    duration = datetime.now() - start
    if filesize > 0:
        with open(filename, "wb") as f:
            f.write(data)
        print(f"got {filesize/1024/1024:7.1f}MB at {filesize / duration.seconds * 8 / 1024 / 1024:5.2f}Mbps saved to {filename}")


Observation    3702107 already downloaded

skipped - got bad request
Observation    7484500 already downloaded
Observation    7505658 already downloaded
Observation    7514494 already downloaded
Observation    7514505 already downloaded
Observation    7514508 already downloaded
Observation   10067782 already downloaded
Observation   10067789 already downloaded
Observation   10279223 already downloaded
Observation   10286988 already downloaded
Observation   10287062 already downloaded
Observation   10287063 already downloaded
Observation   10287064 already downloaded
Observation   10291478 already downloaded
Observation   10291510 already downloaded
Observation   10291512 already downloaded
Observation   10291513 already downloaded
Observation   10349683 already downloaded
Observation   10349685 already downloaded
Observation   10408756 already downloaded
Observation   10408762 already downloaded
Observation   10408763 already downloaded
Observation   10408764 already downloaded
Observation   10408766 already downloaded
Observat